# Dataset

CIFAR 10

In [1]:
import torchvision

cifar10Train = torchvision.datasets.CIFAR10("./CIFAR10", download=True, transform=lambda im: torchvision.transforms.functional.pil_to_tensor(im)/255)
cifar10Test = torchvision.datasets.CIFAR10("./CIFAR10", train=False, download=True, transform=lambda im: torchvision.transforms.functional.pil_to_tensor(im)/255)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import numpy as np
import pandas as pd
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import PIL
from sklearn.metrics import accuracy_score, f1_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models

## CNN4.1

3 Conv, 2 FC, Dropout=0.4, Batch norm

Activation fn: sigmoid

Optimizer: Adam

In [3]:
class CNN4(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn1 = nn.Conv2d(3, 16, (3, 3), padding='same').to(device)
        self.bn1 = nn.BatchNorm2d(16)
        self.sig1 = nn.Sigmoid()
        self.maxpool1 = nn.MaxPool2d((2, 2), stride=(2, 2)).to(device)

        self.cnn2 = nn.Conv2d(16, 32, (3, 3), padding='same').to(device)
        self.bn2 = nn.BatchNorm2d(32)
        self.sig2 = nn.Sigmoid()
        self.maxpool2 = nn.MaxPool2d((2, 2), stride=(2,2)).to(device)

        self.cnn3 = nn.Conv2d(32, 64, (3, 3), padding='same').to(device)
        self.bn3 = nn.BatchNorm2d(64)
        self.sig3 = nn.Sigmoid()
        self.maxpool3 = nn.MaxPool2d((2, 2), stride=(2,2)).to(device)

        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        self.sig4 = nn.Sigmoid()
        self.dropout1 = nn.Dropout(0.4)

        self.linear2 = nn.Linear(512, 512)
        self.sig5 = nn.Sigmoid()
        self.dropout2 = nn.Dropout(0.4)

        self.linear3 = nn.Linear(512, 10)

    def forward(self, inputs):
        x = self.cnn1(inputs)
        x = self.bn1(x)
        x = self.sig1(x)
        x = self.maxpool1(x)

        x = self.cnn2(x)
        x = self.bn2(x)
        x = self.sig2(x)
        x = self.maxpool2(x)

        x = self.cnn3(x)
        x = self.bn3(x)
        x = self.sig3(x)
        x = self.maxpool3(x)

        x = torch.flatten(x, 1)

        x = self.linear1(x)
        x = self.sig4(x)
        x = self.dropout1(x)

        x = self.linear2(x)
        x = self.sig5(x)
        x = self.dropout2(x)

        x = self.linear3(x)
    
        return x


In [4]:
trainloader = DataLoader(cifar10Train, 64)
testloader = DataLoader(cifar10Test, 64)

In [5]:
def train_one_epoch(model, optimzer, loss_fn, training_loader):
    running_loss = 0.
    running_momentum = 0.

    for i, data in enumerate(training_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs.to(device))

        loss = loss_fn(outputs, labels.to(device))
        loss.backward()

        optimizer.step()
           

        running_loss += loss.item()

        
    average_loss = running_loss/(i+1)
    

    return average_loss

In [6]:
def evaluate(tdataset, model):
    model.eval()
    preds = []
    truths = []
    for example in tdataset:
        input = example[0].unsqueeze(0).to(device)
        logits = model(input)
        pred = torch.argmax(torch.softmax(logits, 1))
        preds.append(pred.item())
        truths.append(example[1])
    return accuracy_score(truths, preds), f1_score(truths, preds, average='macro'), preds, truths

In [7]:
def save_model(model, filepath):
    torch.save(model.state_dict(), filepath)

def load_model(model, filepath, device='cpu'):
    model.load_state_dict(torch.load(filepath, map_location=device))
    model.eval()

In [8]:
cnn4 = CNN4().to(device)

optimizer = torch.optim.Adam(cnn4.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [9]:
EPOCHS = 25

total_time = 0

max_f1 = -float('inf')

log = []

for epoch in range(EPOCHS):
    begin = time.time()
    
    cnn4.train(True)
    average_train_loss = train_one_epoch(cnn4, optimizer, loss_fn, trainloader)
    
    end = time.time()
    total_time += end-begin

    cnn4.eval()
    running_test_loss = 0.
    for i, data in enumerate(testloader):
        inputs, labels = data
        outputs = cnn4(inputs.to(device))
        loss = loss_fn(outputs, labels.to(device))
        running_test_loss += loss.item()
    average_test_loss = running_test_loss/(i+1)
    end = time.time()
    if (epoch+1)%1 == 0:
        train_accuracy, train_f1, _, _ = evaluate(cifar10Train, cnn4)
        test_accuracy, test_f1, _, _ = evaluate(cifar10Test, cnn4)
        if test_f1 > max_f1:
            max_f1 = test_f1
            save_model(cnn4, './models/cnn4_1.pth')
        log.append({'average_test_loss': average_test_loss, 'average_train_loss': average_train_loss, 'total_time': total_time, 'train_accuracy': train_accuracy, 'train_f1': train_f1, 'test_accuracy': test_accuracy, 'test_f1': test_f1})
        print(f"Epoch {epoch+1} | train loss: {average_train_loss:.3f} | train accuracy: {100*train_accuracy:.2f}% | train f1: {train_f1:.2f} | test loss: {average_test_loss:.3f} | test accuracy: {100*test_accuracy:.2f}% | test f1: {test_f1:.2f} | time: {total_time:.2f}s")

Epoch 1 | train loss: 1.791 | train accuracy: 38.95% | train f1: 0.35 | test loss: 1.640 | test accuracy: 38.59% | test f1: 0.35 | time: 10.95s
Epoch 2 | train loss: 1.478 | train accuracy: 52.69% | train f1: 0.52 | test loss: 1.288 | test accuracy: 52.40% | test f1: 0.51 | time: 19.80s
Epoch 3 | train loss: 1.315 | train accuracy: 51.05% | train f1: 0.49 | test loss: 1.337 | test accuracy: 50.35% | test f1: 0.49 | time: 28.79s
Epoch 4 | train loss: 1.204 | train accuracy: 56.14% | train f1: 0.56 | test loss: 1.223 | test accuracy: 55.50% | test f1: 0.55 | time: 37.73s
Epoch 5 | train loss: 1.119 | train accuracy: 62.47% | train f1: 0.62 | test loss: 1.063 | test accuracy: 61.45% | test f1: 0.61 | time: 46.74s
Epoch 6 | train loss: 1.050 | train accuracy: 64.23% | train f1: 0.65 | test loss: 1.043 | test accuracy: 63.02% | test f1: 0.64 | time: 55.70s
Epoch 7 | train loss: 0.998 | train accuracy: 63.06% | train f1: 0.63 | test loss: 1.062 | test accuracy: 61.70% | test f1: 0.62 | time:

In [10]:
import pickle

pickle.dump(log, open('./logs/cnn4_1.pkl', 'wb'))